In [2]:
try:
    import geopy # geopy is a library fro geocoding 
except ModuleNotFoundError as e:
    !pip install geopy==2.2.0
    import geopy
if geopy.__version__ != "2.2.0":
    !pip install -U geopy==2.2.0
    import geopy
  
try:
  import geopandas as gpd
except ModuleNotFoundError as e:
  !pip install geopandas==0.10.1
  import geopandas as gpd

if gpd.__version__ != "0.10.1":
  !pip install -U geopandas==0.10.1
  import geopandas as gpd

from geopy.geocoders import Nominatim

     |████████████████████████████████| 118 kB 5.0 MB/s 
  Attempting uninstall: geopy
    Found existing installation: geopy 1.17.0
    Uninstalling geopy-1.17.0:
      Successfully uninstalled geopy-1.17.0


     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 44.6 MB/s 


In [1]:
from google.colab import drive
import os
import json
drive.mount('/content/gdrive') 



Mounted at /content/gdrive


In [4]:
dir = os.listdir('/content/gdrive/MyDrive/KDI_project/JSON_PARSED')

In [23]:
def address_to_latlon(address):
  """
  From an address retrieve the relative 
  coordinates reported in WSG84 reference system (=degree).
  The API provider is ArcGIS.   
  @ return: lat, lon 
  """
  p = gpd.tools.geocode(address, provider="arcgis") 
  return p.geometry.y[0], p.geometry.x[0]




lat, lon = address_to_latlon(address="Via Giuseppe Verdi, 26,  Trento, Italia")

# Puoi scrivere anche Via verdi 26, ma più info gli dai e più precisa sarà la cosa. 
# Perchè ovviamente potrebbe esistere una via verdi 26 da un'altra parte d'Italia o del mondo

Firstly, lat lon is placed for CRUSHSITE events

In [8]:
dir1 = [os.path.join('/content/gdrive/MyDrive/KDI_project/JSON_PARSED', f) for f in dir]

not_successful = []
for file in dir1 : 
  with open(file, encoding='utf-8') as f : 
    try : 
      d = json.load(f)
      try :  
        d['lat_lon'] = address_to_latlon(address=d["location"])
      except :
        not_successful.append(file)
    except : 
      print('Wrong file format {}'.format(file))
      
      

  with open(file, 'w') as f:
    json.dump(d,f)


      

In [34]:
for file in not_successful : 
  with open(file, encoding ='utf-8') as f :
    d = json.load(f) 
    d['location'] = d['location'].replace('(*)','').replace('associativ','').replace('(a maps)','').replace('href=https:','').rstrip('br').rstrip('(a ')
    if ('(a ' in d['location']) :
      l = d['location'].split('(a')
    if ('a //' in d['location']):
      l = d['location'].split('a //')
    else : 
      l = d['location'].split('/')

    if (l[0].lower() != 'addressonline' ) : 
      try : 
        d['lat_lon'] = address_to_latlon(l[0] + ', (Tn)')
      except : 
        print('wrong loc format or encoding {}'.format(file))
      d['info'] += ' '.join(l[1:])
  
  with open(file, 'w') as f:
    json.dump(d,f)
    



    

ESN lat_lon

In [5]:
dir_esn = os.listdir('/content/gdrive/MyDrive/KDI_project/JSON')

In [6]:
dir2 = [os.path.join('/content/gdrive/MyDrive/KDI_project/JSON', f) for f in dir_esn]

not_successful = []
for file in dir2 : 
  with open(file, encoding='utf-8') as f : 
    try : 
      d = json.load(f)
      try :  
        d["location"] = ' '.join(d['location'].lower())
        d['lat_lon'] = address_to_latlon(address=d["location"])
      except :
        not_successful.append(file)
    except : 
      print('Wrong file format {}'.format(file))
      
      

  with open(file, 'w') as f:
    json.dump(d,f)

Wrong file format /content/gdrive/MyDrive/KDI_project/JSON/['C


In [24]:
words = set(['to','on','at','will','before','event?','in','and','decided'])
for file in not_successful : 
  with open(file, encoding ='utf-8') as f :
    d = json.load(f) 
    new = []
    if ('location' in d) and (len(d['location']) > 0):
      d['location'] = ' '.join(d['location']).replace(':','').replace('the',',')
      d['location'] = d['location'].split()

      for w in d['location'] : 
        if (w not in words):
          if (w.lower() not in new) : 
            new.append(w.lower())
      new = ' '.join(new).split(',')
      try : 
        if (len(new) == 1) and (new[0].lower() == 'italy' ):
            d['lat_lon'] = address_to_latlon(new[0] + ', (Tn)')
        else :
          double = []
          for el in new : 
            if ('trento' not in el.lower()) and ('verona' not in el.lower()) \
            and ('bolzano' not in el.lower()) : 
              double.append(address_to_latlon(el + ', (Tn)'))
          d['lat_lon'] = double 
      except :
        print('wrong file format {}'.format(file))

  with open(file, 'w') as f:
    json.dump(d,f)



Stay lat lon

In [26]:
dir_stay = os.listdir('/content/gdrive/MyDrive/KDI_project/JSON (1)')

In [30]:
dir3 = [os.path.join('/content/gdrive/MyDrive/KDI_project/JSON (1)', f) for f in dir_stay]

not_successful = []
for file in dir3 : 
  with open(file, encoding='utf-8') as f : 
    try : 
      d = json.load(f)
      try :  
        d["location"] = ''.join(d['location'].lower())
        d['lat_lon'] = address_to_latlon(address=d["location"])
      except :
        not_successful.append(file)
    except : 
      print('Wrong file format {}'.format(file))
      
      

  with open(file, 'w') as f:
    json.dump(d,f)

In [29]:
print(not_successful)

[]
